In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Div3_Features").getOrCreate()
df = spark.read.parquet("cleaned_fraud_data.parquet")
df.count()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/storage/home/tpk5410/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/storage/home/tpk5410/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
25/12/06 18:37:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/06 18:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/06 18:37:58 WARN Utils: Service 'SparkUI' coul

5998034

In [2]:
# Manual hour binning to avoid interval parsing 
df = df.withColumn("hour_bin", floor(col("Time") / 3600).cast("long"))  

# Window for per hour aggregates
window_agg = Window.partitionBy("hour_bin")

# New features 
df_featured = df.withColumn("tx_velocity", count("row_id").over(window_agg))  # Count tx per hour
df_featured = df_featured.withColumn("amt_per_hour", avg("Amount").over(window_agg))  # Avg amount per hour

# Other transformations 
df_featured = df_featured.withColumn("is_weekend", 
                                    ((floor(col("Time") / 3600) % 24).isin([0, 6, 12, 18])).cast("int"))  # Peak hours flag (0=midnight,6=6am,12=noon,18=6pm; adjust bins if weekends)
df_featured = df_featured.withColumn("v_sum", col("V1") + col("V2") + col("V3"))  # Simple PCA sum

# Global z score: Use dummy partition for dataset wide mean/stddev
global_window = Window.partitionBy(lit(1))  
df_featured = df_featured.withColumn("amount_zscore", 
                                    (col("Amount") - avg("Amount").over(global_window)) / 
                                    (stddev("Amount").over(global_window) + lit(1e-6)))  

# Cache for downstream efficiency
df_featured.cache()
df_featured.count()  

print("Feature engineering complete! New columns added: hour_bin, tx_velocity, amt_per_hour, is_weekend, v_sum, amount_zscore")

25/12/06 18:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/06 18:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/06 18:38:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/06 18:38:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/06 18:38:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/06 18:38:54 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 180.2 MiB so far)
25/12/06 18:38:54 WARN BlockManager: Pers

Feature engineering complete! New columns added: hour_bin, tx_velocity, amt_per_hour, is_weekend, v_sum, amount_zscore


In [3]:
# Validate

print("Updated schema (new cols at end):")
df_featured.printSchema()

print("\nSample rows with new features:")
df_featured.select("Time", "hour_bin", "tx_velocity", "amt_per_hour", "is_weekend", "v_sum", "amount_zscore", "Class").show(10, truncate=False)

# Aggregate insights 
print("\nPer-class feature stats:")
df_featured.groupBy("Class").agg(
    avg("tx_velocity").alias("avg_tx_per_hour"),
    avg("amt_per_hour").alias("avg_amt_per_hour"),
    mean("amount_zscore").alias("mean_zscore"),  
    stddev("amount_zscore").alias("std_zscore")  
).show()

# Global z score check
print("\nGlobal z score check:")
df_featured.select(
    mean("amount_zscore").alias("global_mean"),
    stddev("amount_zscore").alias("global_std")
).show()

Updated schema (new cols at end):
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullab

25/12/06 18:39:39 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 313.2 MiB so far)


+------+--------+-----------+-----------------+----------+--------------------+--------------------+-----+
|Time  |hour_bin|tx_velocity|amt_per_hour     |is_weekend|v_sum               |amount_zscore       |Class|
+------+--------+-----------+-----------------+----------+--------------------+--------------------+-----+
|3421.0|0       |82922      |65.91032693375992|1         |1.750654137015734   |-0.2835590273260809 |0    |
|2802.0|0       |82922      |65.91032693375992|1         |1.25000803300733    |-0.34818900510264644|0    |
|3345.0|0       |82922      |65.91032693375992|1         |2.083606484906916   |0.013221830623907898|0    |
|2941.0|0       |82922      |65.91032693375992|1         |1.107401566872141   |-0.34818900510264644|0    |
|366.0 |0       |82922      |65.91032693375992|1         |1.322487964381769   |0.03549431527306276 |0    |
|2532.0|0       |82922      |65.91032693375992|1         |1.758934130712333   |-0.33033124508930617|0    |
|3308.0|0       |82922      |65.91032

25/12/06 18:39:40 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 313.2 MiB so far)


+-----+------------------+-----------------+--------------------+-----------------+
|Class|   avg_tx_per_hour| avg_amt_per_hour|         mean_zscore|       std_zscore|
+-----+------------------+-----------------+--------------------+-----------------+
|    0|153629.35389505472|88.54972400406585|-2.69639724865191...|0.999806162942079|
|    1|130295.08499257058|86.10998381851581|  0.1599391993425607|1.097461130260703|
+-----+------------------+-----------------+--------------------+-----------------+


Global z score check:


25/12/06 18:39:44 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 313.2 MiB so far)


+--------------------+------------------+
|         global_mean|        global_std|
+--------------------+------------------+
|-1.18386473877399...|0.9999999960228415|
+--------------------+------------------+



In [4]:
# Save featured data
output_path = "featured_fraud_data.parquet"
df_featured.write.mode("overwrite").option("compression", "snappy").parquet(output_path)
print(f"Saved to {output_path} (~{df_featured.count():,} rows, {len(df_featured.columns)} cols)")

# Verify
saved_df = spark.read.parquet(output_path)
print("Saved schema matches:", len(saved_df.columns) == len(df_featured.columns))

25/12/06 18:39:47 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 313.2 MiB so far)
25/12/06 18:40:59 WARN MemoryStore: Not enough space to cache rdd_19_0 in memory! (computed 313.2 MiB so far)


Saved to featured_fraud_data.parquet (~5,998,034 rows, 41 cols)
Saved schema matches: True
